In [2]:
from tqdm import tqdm
import numpy as np
import pymc3 as pm
np.set_printoptions(suppress=True)

Using cuDNN version 7104 on context None
Mapped name None to device cuda0: GeForce GTX 1080 Ti (0000:86:00.0)
/Users/ded/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# Parameters
N = 2000 # True total individuals
S = 10  # Sample periods

# Arrival: 40% initially present
# Add a spike of 30% at time 5
birth = np.concatenate(([.4], np.ones(S-7) * (1-.7)/(S-2), [.3], np.ones(S-5) * (1-.7)/(S-2)), axis=0)

# Departure: fixed 20% rate at each sample
# All depart after the last sample
dep  = np.concatenate((np.random.uniform(low=0.05, high=.15, size=(S-1,)), [1.]), axis=0)

# Capture probability (propensity)
alpha = .12
beta = 8.5
prop = pm.Beta.dist(alpha, beta).random(size = N)

In [4]:
# Capture History Data Structures
CH = [] # Full CH matrix
GCH = [] # First/Last/Count CH matrix
TH = []

# Generate and Arrival Distribution lmd ~ Cat(arr)
ARRIVALS = pm.Categorical.dist(birth).random(size = N)

In [27]:
for i in tqdm(range(N)):
    arrival = ARRIVALS[i]
    dep_tmp = dep.copy()
    dep_tmp[:arrival] = 0
    dep_tmp = np.append([1],(1-dep_tmp).cumprod())[:-1] * dep_tmp # Based on the arrival, compute probs of departure
    departure = pm.Categorical.dist(dep_tmp).random() # Sample departure
    dur = departure - arrival + 1
    capture_history = pm.Bernoulli.dist(prop[i]).random(size=(dur,1))
    capture_history = np.concatenate((np.zeros(arrival), 
                                      np.array(capture_history), 
                                      np.zeros(S - departure - 1)), 
                                     axis = 0)
    TH.append([int(arrival), int(departure), dur, prop[i], capture_history.sum()])
    if capture_history.any():
        first = capture_history.argmax()
        last = S - capture_history[::-1].argmax() -1
        count = capture_history.sum()
        GCH.append([first, last, count])
        CH.append(capture_history)

100%|██████████| 2000/2000 [00:48<00:00, 41.11it/s]


In [28]:
import pickle
fname = 'data/vch-%d-%d-%.2f-%.2f.pkl' % (N, S, alpha, beta)
fname
with open(fname, 'wb') as f:
  pickle.dump((GCH, sorted(TH, key=lambda tup: tup[3])), f)
fname

'data/vch-2000-10-0.12-8.50.pkl'

In [29]:
birth

array([0.4   , 0.0375, 0.0375, 0.0375, 0.3   , 0.0375, 0.0375, 0.0375,
       0.0375, 0.0375])

In [32]:
dep

array([0.11706843, 0.14559058, 0.05254566, 0.12724025, 0.12257379,
       0.09520843, 0.14406404, 0.08546038, 0.07240331, 1.        ])

In [1]:
ARRIVALS

NameError: name 'ARRIVALS' is not defined